In [ ]:
# -*- coding: utf-8
# Repórter Brasil (https://ruralometro2022.reporterbrasil.org.br/)
# Reinaldo Chaves (@paidatocandeira)
# # Procura empresas de deputados federais inscritas em autos de infração do Ibama

In [1]:
import pandas as pd
import unidecode

In [2]:
def f(str):
    try:
        return (unidecode.unidecode(str))
    except:
        return (str)

In [3]:
kwargs = {'sep': ';', 'dtype': str, 'encoding': 'utf-8'}
autos_ibama = pd.read_csv("originais/auto_infracao_ibama_16_fev_2022.csv", **kwargs)
autos_ibama = autos_ibama[pd.notnull(autos_ibama['CPF_CNPJ_INFRATOR'])].copy()
def limpeza1(linha):
    documento = str(linha["CPF_CNPJ_INFRATOR"])
    if len(documento) > 11:
        return documento[:8]
    else:
        return "documento_cpf"

autos_ibama['cnpj_raiz'] = autos_ibama.apply(limpeza1, axis=1)
autos_ibama["nome_infrator_semacentos"] = autos_ibama["NOME_INFRATOR"].str.upper()
autos_ibama["nome_infrator_semacentos"] = autos_ibama["nome_infrator_semacentos"].apply(f)
autos_ibama_pf = autos_ibama[(autos_ibama['cnpj_raiz'] == 'documento_cpf')]
autos_ibama_pj = autos_ibama[(autos_ibama['cnpj_raiz'] != 'documento_cpf')]

In [4]:
kwargs = {'sep': ',', 'dtype': str, 'encoding': 'utf-8'}
empresas_politicos = pd.read_csv("resultados/empresas_deputados/empresas_deputados_fev_2022.csv", **kwargs)

In [5]:
kwargs = {'sep': ',', 'dtype': str, 'encoding': 'utf-8'}
empresas_politicos_socios = pd.read_csv("resultados/empresas_deputados/todos_socios_empresas_deputados_fev_2022.csv", **kwargs)

In [ ]:
# Procura empresas de deputados em autos do Ibama

In [17]:
empresas_politicos.columns

Index(['nome', 'nome_urna', 'politico_id', 'cpf_mascarado', 'cnpj',
       'cnpj_basico_x', 'nome_socio', 'cnpj_cpf_socio', 'nome_fantasia',
       'situacao_cadastral', 'cnae_fiscal', 'cnae_fiscal_secundaria', 'uf',
       'municipio'],
      dtype='object')

In [18]:
autos_ibama_pj.columns

Index(['SEQ_AUTO_INFRACAO', 'NUM_AUTO_INFRACAO', 'SER_AUTO_INFRACAO',
       'TIPO_AUTO', 'TIPO_MULTA', 'VAL_AUTO_INFRACAO', 'PATRIMONIO_APURACAO',
       'GRAVIDADE_INFRACAO', 'UNID_ARRECADACAO', 'DES_AUTO_INFRACAO',
       'DAT_HORA_AUTO_INFRACAO', 'FORMA_ENTREGA', 'DAT_CIENCIA_AUTUACAO',
       'COD_MUNICIPIO', 'MUNICIPIO', 'UF', 'NUM_PROCESSO', 'COD_INFRACAO',
       'DES_INFRACAO', 'TIPO_INFRACAO', 'NOME_INFRATOR', 'CPF_CNPJ_INFRATOR',
       'QTD_AREA', 'INFRACAO_AREA', 'DES_OUTROS_TIPO_AREA',
       'CLASSIFICACAO_AREA', 'NUM_LATITUDE_AUTO', 'NUM_LONGITUDE_AUTO',
       'DES_LOCAL_INFRACAO', 'NOTIFICACAO_VINCULADA', 'ACAO_FISCALIZATORIA',
       'UNID_CONTROLE', 'TIPO_ACAO', 'OPERACAO', 'DENUNCIA_SISLIV',
       'ORDEM_FISCALIZACAO', 'SOLICITACAO_RECURSO', 'OPERACAO_SOL_RECURSO',
       'DAT_LANCAMENTO', 'DAT_ULT_ALTERACAO', 'TIPO_ULT_ALTERACAO',
       'ULTIMA_ATUALIZACAO_RELATORIO', 'cnpj_raiz',
       'nome_infrator_semacentos'],
      dtype='object')

In [19]:
empresas_ibama = pd.merge(empresas_politicos.drop_duplicates('cnpj_basico_x'), autos_ibama_pj.drop_duplicates('cnpj_raiz'), left_on='cnpj_basico_x', right_on='cnpj_raiz', how = 'left')

In [20]:
conta = empresas_ibama[pd.notnull(empresas_ibama['CPF_CNPJ_INFRATOR'])].copy()
conta.shape

(7, 58)

In [21]:
conta.to_excel('resultados/empresas_autos/empresas de deputados em autos do Ibama.xlsx',sheet_name='Sheet1',index=False)

In [22]:
conta.columns

Index(['nome', 'nome_urna', 'politico_id', 'cpf_mascarado', 'cnpj',
       'cnpj_basico_x', 'nome_socio', 'cnpj_cpf_socio', 'nome_fantasia',
       'situacao_cadastral', 'cnae_fiscal', 'cnae_fiscal_secundaria', 'uf',
       'municipio', 'SEQ_AUTO_INFRACAO', 'NUM_AUTO_INFRACAO',
       'SER_AUTO_INFRACAO', 'TIPO_AUTO', 'TIPO_MULTA', 'VAL_AUTO_INFRACAO',
       'PATRIMONIO_APURACAO', 'GRAVIDADE_INFRACAO', 'UNID_ARRECADACAO',
       'DES_AUTO_INFRACAO', 'DAT_HORA_AUTO_INFRACAO', 'FORMA_ENTREGA',
       'DAT_CIENCIA_AUTUACAO', 'COD_MUNICIPIO', 'MUNICIPIO', 'UF',
       'NUM_PROCESSO', 'COD_INFRACAO', 'DES_INFRACAO', 'TIPO_INFRACAO',
       'NOME_INFRATOR', 'CPF_CNPJ_INFRATOR', 'QTD_AREA', 'INFRACAO_AREA',
       'DES_OUTROS_TIPO_AREA', 'CLASSIFICACAO_AREA', 'NUM_LATITUDE_AUTO',
       'NUM_LONGITUDE_AUTO', 'DES_LOCAL_INFRACAO', 'NOTIFICACAO_VINCULADA',
       'ACAO_FISCALIZATORIA', 'UNID_CONTROLE', 'TIPO_ACAO', 'OPERACAO',
       'DENUNCIA_SISLIV', 'ORDEM_FISCALIZACAO', 'SOLICITACAO_RECURS

In [23]:
conta.rename(columns = {'cnpj':'entidade_id'},inplace = True)
conta.rename(columns = {'nome':'politico_nome'},inplace = True)
conta.rename(columns = {'NOME_INFRATOR':'entidade_nome'},inplace = True)

In [24]:
conta = conta[['politico_id', 'entidade_id', 'politico_nome', 'entidade_nome']]

In [25]:
conta.to_csv('resultados/empresas_autos/empresas de deputados em autos do Ibama.csv',index=False)

In [ ]:
# Procura sócio de empresas de deputados em autos do Ibama

In [26]:
empresas_politicos_socios.columns

Index(['cnpj', 'cnpj_basico_x', 'cnpj_basico', 'identificador_de_socio',
       'nome_socio', 'cnpj_cpf_socio', 'qualificacao_socio', 'pais',
       'representante_legal', 'nome_representante'],
      dtype='object')

In [27]:
empresas_politicos_socios['cnpj_cpf_socio']

0          ***564768**
1          ***921272**
2          ***001738**
3          ***381948**
4          ***916458**
             ...      
2096       ***148156**
2097       ***858106**
2098    17845504000105
2099       ***167658**
2100       ***070616**
Name: cnpj_cpf_socio, Length: 2101, dtype: object

In [28]:
 autos_ibama_pf['CPF_CNPJ_INFRATOR']

3         ***909373**
4         ***185942**
5         ***897531**
7         ***768643**
8         ***528073**
             ...     
618511    ***128262**
618512    ***128262**
618513    ***133917**
618514    ***133917**
618517    ***245052**
Name: CPF_CNPJ_INFRATOR, Length: 400169, dtype: object

In [29]:
socios_ibama = pd.merge(empresas_politicos_socios.drop_duplicates('cnpj_cpf_socio'), autos_ibama_pf.drop_duplicates('CPF_CNPJ_INFRATOR'), left_on=['cnpj_cpf_socio', 'nome_socio'], right_on=['CPF_CNPJ_INFRATOR','NOME_INFRATOR'], how = 'left')

In [30]:
conta = socios_ibama[pd.notnull(socios_ibama['CPF_CNPJ_INFRATOR'])].copy()
conta.shape

(24, 54)

In [32]:
conta.to_excel('resultados/empresas_autos/sócios de empresas de deputados em autos do Ibama.xlsx',sheet_name='Sheet1',index=False)

In [33]:
conta.columns

Index(['cnpj', 'cnpj_basico_x', 'cnpj_basico', 'identificador_de_socio',
       'nome_socio', 'cnpj_cpf_socio', 'qualificacao_socio', 'pais',
       'representante_legal', 'nome_representante', 'SEQ_AUTO_INFRACAO',
       'NUM_AUTO_INFRACAO', 'SER_AUTO_INFRACAO', 'TIPO_AUTO', 'TIPO_MULTA',
       'VAL_AUTO_INFRACAO', 'PATRIMONIO_APURACAO', 'GRAVIDADE_INFRACAO',
       'UNID_ARRECADACAO', 'DES_AUTO_INFRACAO', 'DAT_HORA_AUTO_INFRACAO',
       'FORMA_ENTREGA', 'DAT_CIENCIA_AUTUACAO', 'COD_MUNICIPIO', 'MUNICIPIO',
       'UF', 'NUM_PROCESSO', 'COD_INFRACAO', 'DES_INFRACAO', 'TIPO_INFRACAO',
       'NOME_INFRATOR', 'CPF_CNPJ_INFRATOR', 'QTD_AREA', 'INFRACAO_AREA',
       'DES_OUTROS_TIPO_AREA', 'CLASSIFICACAO_AREA', 'NUM_LATITUDE_AUTO',
       'NUM_LONGITUDE_AUTO', 'DES_LOCAL_INFRACAO', 'NOTIFICACAO_VINCULADA',
       'ACAO_FISCALIZATORIA', 'UNID_CONTROLE', 'TIPO_ACAO', 'OPERACAO',
       'DENUNCIA_SISLIV', 'ORDEM_FISCALIZACAO', 'SOLICITACAO_RECURSO',
       'OPERACAO_SOL_RECURSO', 'DAT_LAN

In [34]:
conta.rename(columns = {'cnpj_cpf_socio':'entidade_id'},inplace = True)
conta.rename(columns = {'nome_socio':'entidade_nome'},inplace = True)

In [35]:
conta = conta[['entidade_id', 'entidade_nome']]

In [36]:
conta.to_csv('resultados/empresas_autos/sócios de empresas de deputados em autos do Ibama.csv',index=False)